# SDF and Gaia - a new world

In [ ]:
# For rendering code snippets, SVG
from IPython.display import HTML, Code, SVG

import pprint

# A hack to get Code syntax highlighting to work in Jupyterlab
# https://github.com/ipython/ipython/issues/11747#issuecomment-528694702
def display_source(code, language : str = "python3", style="material"):
    def _jupyterlab_repr_html_(self):
        from pygments import highlight
        from pygments.formatters import HtmlFormatter

        fmt = HtmlFormatter(style=style)
        style_outer = "<style>{}\n{}</style>".format(
            fmt.get_style_defs(".output_html"), fmt.get_style_defs(".jp-RenderedHTML")
        )
        return style_outer + highlight(self.data, self._get_lexer(), fmt)

    # Replace _repr_html_ with our own version that adds the 'jp-RenderedHTML' class
    # in addition to 'output_html'.
    Code._repr_html_ = _jupyterlab_repr_html_
    return Code(data=code, language=language)

# Note: this code assumes real_world environment is sourced.

## Unified World Model

We want a single modular description format that can naturally describe the visual, physical and logical aspects (e.g. base configuration, virtual walls) of a complete robot line and all relevant objects for both real and simulated systems. This will allow us to have a single place where we can store this information (reducing complexity and error surface) and maximize reuse of tooling and assets.

Our current solution is far from unified:

- RobotPlanningScene with simple blocky parts we care about for (planning) collision checking.
- Robot URDF.xacro + SRDF + EI Description
- No concept of a "world" - everything expressed in "the" robot's base frame. 
- Inflexible, brittle, not scalable (e.g. robots on slides, multiple actuatable models in a world, external sensors, tooling to visualize/query all geometric information).

## SDF

For the Simulation stack we need to represent the world, along with all properties we care about. The format it uses (on disk) is [SDF](http://sdformat.org/) (Simulation Description Format). It is a powerful format for describing models, frames, and how they compose into worlds. Under active development and used by multiple teams ([OSRF](https://www.openrobotics.org/), [Drake](https://drake.mit.edu/)). 

We are transitioning to using SDF for describing our real world systems. Mixed Palletizing Bot will be the first application to use SDF from the ground up.

### SDF Examples

#### A simple world

Worlds are formed by the composition of frames and models.

A bare bones example of a world with one model and one explicit frame:

In [ ]:
display_source("simple_world.sdf", language="xml")

In [ ]:
%%bash --no-raise-error
# Visualize it in GV - you can also spin up an ignition instance, but that's much slower :)
rosrun gaia gv simple_world.sdf

Points of attention:

- A World composes a collection of models and frames.

#### The anatomy of a model

Let's look at a more interesting model with links and joints. 


In [ ]:
display_source("model_with_joints.sdf", language="xml")

In [ ]:
%%bash --no-raise-error
# Visualize it in GV
rosrun gaia gv model_with_joints.sdf

#### A realistic example: Mixed Palletizing Bot dev cell

Let's look at the Mixed Palletizing Bot dev cell world for a real world example.

In [ ]:
# Atlas contains SDF worlds and models
from atlas.resolvers import resolve_sdf_world_path

mpb_dev_cell_sdf_path = resolve_sdf_world_path("mpb_dev_cell.sdf")

display_source(mpb_dev_cell_sdf_path, language="xml")

Some points of attention:

- Includes and `model://` 
- Workspaces


In [ ]:
%%bash --no-raise-error -s "$mpb_dev_cell_sdf_path"
# Visualize it in GV
rosrun gaia gv $1

### Vicarious custom SDF extensions

We have a few custom elements in SDF to deal with information like "workspaces" and "planning volumes" (see also the MPB visualization above).

You can read more about them here:

[Vicarious custom SDF extensions](http://docs.corp.vicarious.com/master/common/sdf/doc/user/custom_sdf_elements.html)


### What about URDF(.xacro)?

[URDF](http://wiki.ros.org/urdf) (Universal Robot Description Format) is the format we currently use to represent robot models. It is strictly less powerful than SDF: anything that can be expressed in URDF can be expressed in SDF, but not vice versa. 

We will move to using SDF robot models as the ground truth, and generate URDF models on the fly, where needed (e.g. RPS, KDL).

External cameras now (hackily) defined in robot models will move to the `<world>` as stand alone `<model>`s.

We aim to complete the switch to SDF for all suction robots in 2021-Q3.


## Gaia 

Gaia is an in-house solution for representing (robot) models and frames, and it can load SDF. It comes with a handy viewer `gv` which you already saw in action. It is designed to be flexible (quickly add new information you want to represent) and scalable (it can work for entire robot lines).

The current use is to get information about the world that is statically known (e.g. where the BLB is in the robot base frame) for simplifying app configuration. We are working on extending it to deal with dynamic information (e.g. joint angles, pose detections) and show a globally consistent world view.


### Demo - exploring Mixed Palletizing Bot dev cell 


In [ ]:
from gaia.gaia import Gaia

# Load the world
world = Gaia(mpb_dev_cell_sdf_path)

In [ ]:
# What (top level) models does it have?

world.get_model_names() 

In [ ]:
# Getting transforms - works for any implicit (links, visuals, etc.) or explicit <frame>
robot_in_world = world.get_transform(source_frame="main_arm/arm/base_link")
print(robot_in_world)

# Everything works with scoped paths, and these directly follow the SDF model structure

In [ ]:
# Where is the pallet-right in arm's frame?
pallet_right_in_robot =  world.get_transform(source_frame="pallet-right", target_frame="main_arm/arm/base_link")
print(pallet_right_in_robot)

In [ ]:
# Get workspace geometry (scoped path into a model - link - visual)

workspace_shape = world.get_geometry_shape("conveyor-workspace/body/workspace")
workspace_in_world = world.get_transform("conveyor-workspace/body/workspace")

print(workspace_shape)
print(workspace_in_world)

In [ ]:
# Get joint angles for any actuatable model

# By convention, "the" robot is called "main_arm"
robot_joint_positions = world.get_model_joint_positions("main_arm")
pprint.pprint(robot_joint_positions)

In [ ]:
# Set them. You can provide a nested model path as well. 
# Only need to provide the joints you want to set.
world.set_model_joint_positions("main_arm/arm", [('shoulder_pan_joint', 1.31)])


robot_joint_positions = world.get_model_joint_positions("main_arm")
pprint.pprint(robot_joint_positions)

In [ ]:
# Of course, transforms reflect moving models. Gaia guarantees a consistent view.

pallet_left_in_gripper = world.get_transform(source_frame="pallet-left", target_frame="main_arm/arm/gripper_tip")

print("Before move ", pallet_left_in_gripper)


world.set_model_joint_positions("main_arm/arm", [('shoulder_pan_joint', 1.)])


pallet_left_in_gripper = world.get_transform(source_frame="pallet-left", target_frame="main_arm/arm/gripper_tip")

print("After move ", pallet_left_in_gripper)

These are the main use cases today.


You can learn more in the [documentation](http://docs.corp.vicarious.com/master/platform/gaia/doc/overview.html) and [tests]()



## Interactions with Robot Planning Scene (RPS), GeometricScene?

These are more specialized world models - they care only about a subset of the information (e.g. planning geometry) and need extra functionality (e.g. fast collision checking).


They will be directly instantiatable from a Gaia instance (directly in C++)

## Future

- In the (very) near future: query `<sensor>` and automatically configure Ignition cameras from SDF. Visualize sensor frames and frustrum (where applicable) in GV.
- Dynamic Gaia: a live view of all geometry animated with joint updates, model moves. One server with client instances that get (configurable) updates. Also still possble to have stand alone instances for subsets of data you care about (Mini Gaia).
- Instantiate `RobotPlanningScene` directly from Gaia. Pick your slice of interest (e.g. robot x and everything within three meters around it).
- SDF layering: One layer to represent physical systems, other layers for application specific information such as workspaces and special frames (e.g. a pregrasp frame)
- Better tooling for SDF manipulation/templating. Currently we have an [`sdf`](https://github.com/vicariousinc/real_world/tree/master/common/sdf) library to do some very light work, but it doesn't completely support the current SDF version and the API is pretty unfriendly. Ideally we want something more friendly than XACRO for procedurally generated models and worlds.
- Collision checking layer (much like how `GV` is a layer on top of the core Gaia data structures), which will give us much more control over the API and backend (e.g. [FCL](https://github.com/flexible-collision-library/fcl), [Bullet](https://pybullet.org/Bullet/BulletFull/classbtCollisionWorld.html), which versions to use) than with RobotPlanningScene, which is rooted in [MoveIt!](https://moveit.ros.org/). 

In [ ]:
# A rough overview of SDF / Gaia dataflow.
SVG(filename="data_flow.dio.svg")

## Further reading

- SDF specification. What is a [`<model>`](http://sdformat.org/spec?ver=1.8&elem=model)? What is a [`<link>`](http://sdformat.org/spec?ver=1.8&elem=link)? Read it here.
- SDFormat features: 
    - [Collisions and Visuals](http://sdformat.org/tutorials?tut=spec_shapes&cat=specification&). 
    - [Pose Frame Semantics](http://sdformat.org/tutorials?tut=pose_frame_semantics_proposal&cat=pose_semantics_docs&). This devlves into detail into the differences with URDF.
    - [Model composition](http://sdformat.org/tutorials?tut=composition_proposal&cat=pose_semantics_docs&). Models can be composed out of other models. In the future this will allow us to eliminate most uses of XACRO, as it is often used for composition.
    - [Custom SDF extensions](http://sdformat.org/tutorials?tut=custom_elements_attributes_proposal&cat=pose_semantics_docs&). Read this to understand the vicarious custom elements (see below).
- [Vicarious custom SDF extensions](http://docs.corp.vicarious.com/master/common/sdf/doc/user/custom_sdf_elements.html)
- [Gaia documentation](http://docs.corp.vicarious.com/master/platform/gaia/doc/overview.html)
